# Part 1

Local optimum only (with randomized starting states).

In [1]:
filename = "inputs/12-16.txt"
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re
import networkx as nx

In [3]:
flows = {}
G = nx.Graph()
for l in data.splitlines():
    m = re.match("Valve ([A-Z]+) has flow rate=(\d+); tunnels? leads? to valves? (.*)", l)
    valve, flow, neighbors = m.groups()
    for n in neighbors.split(", "):
        G.add_edge(valve, n)
    flows[valve] = int(flow)

In [4]:
def flow(order, graph, flows, time=30, start="AA"):
    pos = start
    opened = []
    pressure = 0
    for target in order:
        time -= nx.shortest_path_length(G, pos, target) + 1
        if time <= 0:
            break
        pressure += flows[target] * time
        pos = target
    return pressure

In [5]:
import random

In [6]:
candidates = [v for v in flows if flows[v] > 0]
iorder = candidates.copy()
imax_flow = flow(candidates, G, flows)
when = None
for iter_ in range(100):
    max_flow = flow(candidates, G, flows)
    better = True
    order = candidates.copy()
    random.shuffle(order)
    while better:
        better = False
        # try to move the i-th valve to the j-th position
        for i in range(len(candidates)):
            for j in range(len(candidates) - 1):
                cand = order.copy()
                cand.insert(j, cand.pop(i))
                fl = flow(cand, G, flows)
                if fl > max_flow:
                    order = cand
                    better = True
                    max_flow = fl
    if max_flow > imax_flow:
        iorder = order
        imax_flow = max_flow
        when = iter_
print(imax_flow, iorder, when)

1880 ['OM', 'VR', 'SP', 'RO', 'KZ', 'DI', 'SO', 'SC', 'AJ', 'JD', 'PW', 'IR', 'VG', 'JL', 'RI'] 0


# Part 2

Local optimum only (with randomized starting states).

In [7]:
def flow(orders, spt, graph, flows, st_time=26, start="AA"):
    if not 0 <= spt < len(orders):
        return 0
    pressure = 0
    for order in (orders[:spt], orders[spt:]):
        time = st_time
        pos = start
        opened = []
        for target in order:
            time -= nx.shortest_path_length(G, pos, target) + 1
            if time <= 0:
                break
            pressure += flows[target] * time
            pos = target
    return pressure

In [8]:
candidates = [v for v in flows if flows[v] > 0]
iorder = candidates
ispt = 0
imax_flow = flow(candidates, 0, G, flows)
when = None
for iter_ in range(100):
    better = True
    order = candidates.copy()
    random.shuffle(order)
    spt = random.randrange(len(candidates))
    max_flow = flow(order, spt, G, flows)
    while better:
        better = False
        # try to move the i-th valve to the j-th position
        for i in range(len(candidates)):
            for j in range(len(candidates) - 1):
                cand = order.copy()
                cand.insert(j, cand.pop(i))
                for s in (spt - 1, spt, spt + 1):
                    fl = flow(cand, s, G, flows)
                    if fl > max_flow:
                        spt = s
                        order = cand
                        better = True
                        max_flow = fl
    if max_flow > imax_flow:
        iorder = order
        imax_flow = max_flow
        ispt = spt
        when = iter_
print(imax_flow, iorder, ispt, when)

2520 ['OM', 'VR', 'SP', 'RO', 'KZ', 'DI', 'SO', 'SC', 'PW', 'JL', 'IR', 'VG', 'AJ', 'RI', 'JD'] 8 4
